In [1]:
using Pkg, Gurobi, JuMP, CSV, DataFrames, LinearAlgebra, Statistics, Plots

## Formulation:

![alt text](Network_Diagram.jpg "Network Diagram")


In [39]:
chemicals = CSV.read("chemicals.csv", DataFrame)
p = size(chemicals)[2]
# print(chemicals)
p_matrix = Matrix(hcat(Vector(chemicals[2, 2:p]), Vector(chemicals[3, 2:p]), Vector(chemicals[5, 2:p]), Vector(chemicals[4, 2:p]), Vector(chemicals[1, 2:p]))')

5×4 Matrix{Float64}:
 128.98  0.00012898   449.0  0.45
  40.0   4.0e-5       492.0  0.49
  58.44  5.844e-5       0.0  0.0
  18.0   1.8e-5         0.0  0.0
  92.52  9.252e-5    1840.0  1.84

In [40]:
customers = CSV.read("customers.csv", DataFrame)
k_matrix = Matrix(customers)

Row,Price (USD/MT),Demand,Purity
,Int64,Int64,Float64
1,1840,5000,0.15
2,2000,3000,0.12
3,4000,1000,0.05
4,1500,8000,0.14
5,1700,8000,0.17
6,3000,5000,0.08


In [45]:
reactors = CSV.read("reactors.csv", DataFrame)
r_matrix = Matrix(reactors)

5×4 Matrix{Float64}:
 500000.0    100.0  10000.0  0.9
 800000.0    175.0  10000.0  0.95
      1.0e6  200.0  10000.0  0.99
 300000.0     80.0  10000.0  0.8
 400000.0     85.0  10000.0  0.88

In [44]:
separators_1 = CSV.read("separators1.csv", DataFrame)
m_matrix = Matrix(separators_1)

5×4 Matrix{Float64}:
 100000.0   50.0  10000.0  0.05
 200000.0  100.0  20000.0  0.01
 250000.0  120.0  21000.0  0.005
  50000.0   35.0  70000.0  0.2
  75000.0   45.0  80000.0  0.1

In [47]:
separators_2 = CSV.read("separators2.csv", DataFrame)
s_matrix = Matrix(separators_2)

5×4 Matrix{Float64}:
 100000.0  50.0  10000.0  0.9
 200000.0  50.0  10000.0  0.95
 250000.0  50.0  10000.0  0.99
  50000.0  50.0  10000.0  0.85
  75000.0  50.0  10000.0  0.88

In [53]:
suppliers_DCH = CSV.read("suppliers_DCH.csv", DataFrame)
n_p1_matrix = Matrix(suppliers_DCH)

5×3 Matrix{Float64}:
  450.0  10000.0  0.05
  500.0  10000.0  0.04
 1000.0   5000.0  0.005
  250.0  10000.0  0.2
  750.0   5000.0  0.02

In [54]:
suppliers_NAOH = CSV.read("suppliers_NAOH.csv", DataFrame)
n_p2_matrix = Matrix(suppliers_NAOH)

5×3 Matrix{Float64}:
  500.0  2500.0  0.05
  600.0  2000.0  0.04
  400.0  3000.0  0.07
 1000.0  2000.0  0.005
  800.0  2500.0  0.01

In [55]:
wastes = CSV.read("wastes.csv", DataFrame)
# j = wastes[1, 1]
# l_aqueous = wastes[1, 2]
# l_organic = wastes[1, 3]

Row,W1 Cost per MT,W2 Cost per Aqeuous MT,W2 Cost per Organic MT
,Int64,Int64,Int64
1,100,100,500


\begin{align}
    \text{Sets of Values:}& \\
    \text{Supplier ($n$)}& \in [1, N] \\
    \text{Material ($p$)}& \in [1, P] \\
    \text{$p=1$ for DCH, $p=2$ for NaOH, $p=3$ for NaCl, $p=4$ for H2O, } & \text{and $p=5$ ECH} \\
    \text{Separator (Left) ($m$)}& \in [1, M] \\
    \text{Waste Water Treatment Plant (for initial separators) ($j$)}& \in [J] \\
    \text{Reactors ($r$)}& \in [1, R] \\
    \text{Separator (Right) ($s$)}& \in [1, S] \\
    \text{Customers ($k$)}& \in [1, K] \\
    \text{Waste Water Treatment Plant (for second separators) ($l$)}& \in [L] \\ \\
    \text{Fixed Cost for using separator (left) ($m$)} & = f_{m} \\
    \text{Fixed Cost for using reactor ($r$)} & = f_{r} \\
    \text{Fixed Cost for using separator (right) ($s$)} & = f_{s} \\ \\
    \text{Unit Cost for sending material to separator (left) ($m$)} & = c_{m} \\
    \text{Unit Cost for sending material to waste water treatment plant ($j$)} & = c_{j} \\
    \text{Unit Cost for sending material to reaction ($r$)} & = c_{r} \\
    \text{Unit Cost for sending material to separator (right) ($s$)} & = c_{s} \\
    \text{Unit Cost for sending material to waste water treatment plant (aqueous) ($l$)} & = c^{A}_{l} \\
    \text{Unit Cost for sending material to waste water treatment plant (organic) ($l$)} & = c^{O}_{l} \\
\end{align}

\begin{align}
    \text{Variables and Sign Restrictions:} \\
    a_{nm}^{p} \in \mathbb{R}_{+} =&\ \text{amount of material $p$ purchased from supplier $n$ and sent to separator $m$} \\
    I_{n} =&\ \text{\% impurity in material $p$ purchased from supplier $n$} \\ \\
    b_{mr}^{p} \in \mathbb{R}_{+} =&\ \text{amount of separated (pure) material $p$ to send from separator $m$ to reactor $r$} \\
    \Delta b_{mj}^{p} \in \mathbb{R}_{+} =&\ \text{amount of separated (impure) material $p$ to send from separator $m$ to waste plant $j$} \\ \\
    d_{rs}^{p} \in \mathbb{R}_{+} =&\ \text{amount of reacted material $p$ (with impurities) to ship from reactor $r$ to separator $s$} \\
    \gamma_{r} =&\ \text{\% yield during reaction from reactor $r$} \\ \\
    e_{sk}^{p} \in \mathbb{R}_{+} =&\ \text{amount of separated material $p$ (pure) to ship from separator $s$ to customer $k$} \\
    \Delta e_{sl}^{A} \in \mathbb{R}_{+} =&\ \text{amount of separated (impure) aqueous material to ship from separator $s$ to waste plant $l$} \\
    \Delta e_{sl}^{O} \in \mathbb{R}_{+} =&\ \text{amount of separated (impure) organic material to ship from separator $s$ to waste plant $l$} \\ 
    I_{s} =&\ \text{\% recovery of materials from separator $s$} \\ \\
    g_{k} \in \mathbb{R}_{+} =&\ \text{units of demand from customer k} \\ 
    \nu_{k} =&\ \text{maximum \% of non-ECH product that customer $k$ willing to except, relative to amount of ECH shipped}\\ \\
    \rho_{m} \in \{0, 1\} =&\ \text{binary indicator if separator (left) $m$ used} \\
    \sigma_{r} \in \{0, 1\} =&\ \text{binary indicator if reactor $r$ used} \\
    \mu_{s} \in \{0, 1\} =&\ \text{binary indicator if separator (right) $s$ used} \\
\end{align}

In [ ]:
model = Model(Gurobi.Optimizer)

## Existing Data

# p_matrix: 5x4 matrix 
    # Rows: DHC, NaOH, NaCl, H2O, ECH (material index)
    # Columns: [1] MW (g/mol), [2] MW (MT/mol), [3] Price (USD/MT), [4] Price (USD/kg)
# k_matrix: 6x2 matrix
    # Rows: Customer index
    # Columns: [1] Price (USD/MT), [2] Demand, [3] Purity
# r_matrix: 5x4 matrix
    # Rows: Reactor index
    # Columns: [1] Fixed Cost, [2] Flow Cost, [3] Capacity, [4] Conversion
# m_matrix: 5x4 matrix [Left Separators]
    # Rows: Separator (Left) index
    # Columns: [1] Fixed Cost, [2] Flow Cost, [3] Capacity, [4] Impurity %
# s_matrix: 5x4 matrix [Right Separators]
    # Rows: Separator (Right) index
    # Columns: [1] Fixed Cost, [2] Flow Cost, [3] Capacity, [4] Recovery
# n_p1_matrix: 5x3 matrix (DCH Suppliers)
    # Rows: DCH Supplier index
    # Columns: [1] Price ($/MT), [2] Supply, [3] Impurity %
# n_p1_matrix: 5x3 matrix (NaOH Suppliers)
    # Rows: NaOH Supplier index
    # Columns: [1] Price ($/MT), [2] Supply, [3] Impurity %
# j: Int64 (Unit Cost of waste water treatment at left separators)
# l_aqueous: Int64 (Unit Cost of aqueous water water treatment disposal at right separators)
# l_organic: Int64 (Unit Cost of organic waste water treatment disposal at right separators)

## Ranges & Values


\begin{align}
    & \text{Objective Function:}& \\
    \max & \quad R_{k} - (FC_{m} + UC_{m}) - (FC_{r} + UC_{r}) - (FC_{s} + UC_{s}) - UC_{j} - UC_{l}
\end{align}

\begin{align}
    \text{Flow Constraints (MT units):}& \\
        &\sum_{n=1}^{N} a_{nm}^{p} = \sum_{j=1}^{J} \Delta b_{mj}^{p} + \sum_{r=1}^{R} b^{p}_{mr}\ , \quad \forall p \in \{1, 2\}, \forall m \in M, \forall j \in J \\
        &\sum_{m=1}^{M} \sum_{p=1}^{P} b_{mr}^{p} = \sum_{s=1}^{S}d_{rs}^{p}\ , \quad \forall r \in R, \forall p \in \{1, 2\} \\
        &\sum_{r=1}^{R} d_{rs}^{p} = \sum_{l=1}^{L} \Delta e_{sl}^{p} + \sum_{k=1}^{K} e_{sk}^{p}\ , \quad \forall s \in S, \forall p \in P \\
        &a_{nm}^{p} = b_{mr}^{p} = \Delta b_{mj}^{p} = 0\ , \quad \forall p \in \{3, 4, 5\}, \forall n \in N, \forall m \in M \\ \\ 
    \text{Fixed Cost for Left Separators:}& \\
        &a_{nm}^{p} \leq \mathbf{M} \times \rho_{m}\ , \quad \forall n \in N, \forall p \in P, \forall m \in M \\
        &b_{mr}^{p} \leq \mathbf{M} \times \rho_{m}\ , \quad \forall p \in P, \forall m \in M, \forall r \in R \\
        &\Delta b_{mj}^{p} \leq \mathbf{M} \times \rho_{m}\ , \quad \forall p \in P, \forall m \in M, \forall j \in J \\
        &FC_{m} = \sum_{m=1}^{M} f_{m} \times \rho_{m} \\
    \text{Fixed Cost for Reactors:}& \\
        &b_{mr}^{p} \leq \mathbf{M} \times \sigma_{r}\ , \quad \forall p \in P, \forall m \in M, \forall r \in R \\
        &d_{rs}^{p} \leq \mathbf{M} \times \sigma_{r}\ , \quad \forall r \in R, \forall p \in P, \forall s \in S \\
        &FC_{r} = \sum_{r=1}^{R} f_{r} \times \sigma_{r} \\ \\
    \text{Fixed Cost for Right Separators:}& \\
        &d_{rs}^{p} \leq \mathbf{M} \times \mu_{s}\ , \quad \forall r \in R, \forall p \in P, \forall s \in S \\
        &e_{sk}^{p} \leq \mathbf{M} \times \mu_{s}\ , \quad \forall s \in S, \forall p \in P, \forall k \in K \\
        &\Delta e_{sl}^{p} \leq \mathbf{M} \times \mu_{s}\ , \quad \forall s \in S, \forall p \in P, \forall l \in L \\
        &FC_{s} = \sum_{s=1}^{S} f_{r} \times \mu_{s} \\ \\
    \text{Demand Constraints:}& \\
        &\sum_{s=1}^{S} e_{sk}^{5} \leq g_{k}\ , \quad \forall k \in K \\ 
        &\sum_{s=1}^{S} \sum_{p=1}^{4} e_{sk}^{p} \leq \nu_{k} \times \sum_{s=1}^{S} e_{sk}^{5}\ , \quad \forall k \in K\\
    \text{Calculate Unit Costs:}& \\
        &UC_{m} = \sum_{p=1}^{P} \sum_{n=1}^{N}\sum_{m=1}^{M} c_{m} \times a_{nm}^{p} \\
        &UC_{j} = \sum_{p=1}^{P} \sum_{m=1}^{M} \sum_{j=1}^{J} c_{j} \times \Delta b_{mj}^{p} \\
        &UC_{r} = \sum_{p=1}^{P} \sum_{m=1}^{M}\sum_{r=1}^{R} c_{r} \times b_{mr}^{p} \\
        &UC_{s} = \sum_{p=1}^{P} \sum_{r=1}^{R} \sum_{s=1}^{S} c_{s} \times d_{rs}^{p} \\
        &UC_{l}= \sum_{s=1}^{S} \sum_{l=1}^{L} (c_{l}^{A} \times \Delta e_{sl}^{A} + c_{l}^{O} \times \Delta e_{sl}^{O}) \\ \\
    \text{Capacity Constriants:}& \\
        &\sum_{m=1}^{M} a_{nm}^{p} \leq supply_{N}\ , \quad \forall n \in N, \forall p \in \{1, 2\} \\
        &\sum_{n=1}^{N} \sum_{p=1}^{2} a_{nm}^{p} \leq capacity_{M}\ , \quad \forall m \in M \\
        &\sum_{j=1}^{J} \Delta b_{mj}^{p} + \sum_{r=1}^{R} \sum_{p=1}^{2} b_{mr}^{p} \leq capacity_{M}\ , \quad \forall m \in M \\
        &\sum_{m=1}^{M} \sum_{p=1}^{2} b_{mr}^{p} \leq capacity_{R}\ , \quad \forall r \in R \\
        &\sum_{s=1}^{S} \sum_{p=1}^{2} d_{rs}^{p} \leq capacity_{R}\ , \quad \forall r \in R \\
        &\sum_{r=1}^{R} \sum_{p=1}^{P} d_{rs}^{p} \leq capacity_{S}\ , \quad \forall s \in S \\
        &\sum_{l=1}^{L} \sum_{p=1}^{P} \Delta e_{sl}^{p} + \sum_{p=1}^{P} \sum_{k=1}^{K} e_{sk}^{p} \leq capacity_{S}\ , \quad \forall s \in S \\
\end{align}

\begin{align}
    \text{Calculations (Suppliers $\rightarrow$ Left Separators):}& \\
        &\sum_{n=1}^{N} (1 - I_{n}) \times a_{nm}^{p} = \sum_{r=1}^{R} b_{mr}^{p}\ , \quad \forall m \in M, \forall p \in \{1, 2\} \\
        &\sum_{n=1}^{N} I_{n} \times a_{nm}^{p} = \sum_{j=1}^{J} \Delta b_{mj}^{p}\ , \quad \forall m \in M, \forall p \in \{1, 2\} \\ \\
    \text{Calculations (Left Separators $\rightarrow$ Reactors):}& \\
        &\sum_{m=1}^{M} MW_{p=1} \times (\frac{b_{mr}^{p=1}}{MW_{p=1}} - \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=1}\ , \quad \forall r \in R \\
        &\sum_{m=1}^{M} MW_{p=2} \times (\frac{b_{mr}^{p=2}}{MW_{p=2}} - \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=2}\ , \quad \forall r \in R \\
        &\sum_{m=1}^{M} (MW_{p=3} \times \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=3}\ , \quad \forall r \in R \\
        &\sum_{m=1}^{M} (MW_{p=4} \times \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=3}\ , \quad \forall r \in R \\
        &\sum_{m=1}^{M} (MW_{p=5} \times \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=3}\ , \quad \forall r \in R \\ \\
    \text{Calculations (Reactors $\rightarrow$ Right Separators):}& \\
        &\sum_{r=1}^{R} I_{s} \times d_{rs}^{p} = \sum_{l=1}^{L} \Delta e_{sl}^{p}\ , \quad \forall s \in S, \forall p \in \{1, 2, 3, 4\} \\
        &\sum_{r=1}^{R} d_{rs}^{5} = \sum_{k=1}^{K} e_{sk}^{5}\ , \quad \forall s \in S \\
        &\sum_{r=1}^{R} (1 - I_{s}) \times d_{rs}^{p} = \sum_{k=1}^{K} e_{sk}^{p}\ , \quad \forall s \in S, \forall p \in \{1, 2, 3, 4\}
\end{align}